In [47]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.exceptions import SpotifyException
import pandas as pd
import time

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = 'fb35e1e6b7c54dc6b625996ae72f9f81',
                                                          client_secret = '72eeda8532044330ba8a17eacdea3e62'))




In [2]:
results['tracks']['items'][0]['album']['artists'][0]['name']

all_artist_names = []

for track in results['tracks']['items']:
    for artist in track['album']['artists']:
        all_artist_names.append(artist['name'])

print(all_artist_names)

['Drake', 'SZA', 'Drake', '21 Savage', 'Drake', 'Drake', 'Drake']


### Song catcher 
- Step one
build function to scrape artists from a playlist, and store them in a list
spotipy used: sp.user_playlist_tracks('spotify', 'playlist Uri')

- step two
build function that takes the list and returns 30 songs from the artist and stores them in a list with the Uris
spotipy used: sp.search(q = 'artist : {artist}, limit = 30, type = 'track')
might have to add a failsafe if artist doesnt have 30 songs

- step three
use Uris for each track Uris to get audio features
spotipy used : sp.audio_features


In [3]:
#Scrape artists from spotify and store in a list
def artists_spot(playlist_id):
    
    all_artist_names = []

    results = sp.user_playlist_tracks('spotify', playlist_id )
    #print("Initial Results:", results)
    tracks = results['items']

    for track in tracks:
        #append artist names to list
        for artist in track['track']['album']['artists']:
            all_artist_names.append(artist['name'])
        
        
    while results['next']:
        #handling pages
        results = sp.next(results)
        tracks = results['items']
        for track in tracks:
            for artist in track['track']['album']['artists']:
                all_artist_names.append(artist['name'])
                
                
    return all_artist_names
            
            
    
    

In [4]:
 playlist_id = '37i9dQZF1DX4o1oenSJRJd'

In [5]:
 artists_spot(playlist_id)

['Coldplay',
 'Eminem',
 'Kanye West',
 'Linkin Park',
 'The Killers',
 'Shakira',
 'The White Stripes',
 'Daddy Yankee',
 'Coldplay',
 'Linkin Park',
 'Eminem',
 'Kanye West',
 'Nelly Furtado',
 'Rihanna',
 'Coldplay',
 'Gorillaz',
 'Evanescence',
 'Eminem',
 'Foster The People',
 'Train',
 'Akon',
 'JAY-Z',
 'Coldplay',
 'USHER',
 'Kings of Leon',
 'Kesha',
 'Outkast',
 'Red Hot Chili Peppers',
 'Timbaland',
 'Jimmy Eat World',
 'Kanye West',
 'Lady Gaga',
 'Coldplay',
 'The Game',
 'Eminem',
 'Hoobastank',
 'Outkast',
 'Papa Roach',
 'Sean Paul',
 'Snow Patrol',
 'Nelly Furtado',
 'Maroon 5',
 'Nickelback',
 'Flo Rida',
 'Kendrick Lamar',
 'Kanye West',
 'Dr. Dre',
 'Nelly Furtado',
 'Black Eyed Peas',
 'Alicia Keys',
 '50 Cent',
 'Empire of the Sun',
 'Britney Spears',
 'The Fray',
 'Rihanna',
 'Kanye West',
 'Maroon 5',
 'Avril Lavigne',
 'Franz Ferdinand',
 'Justin Timberlake',
 'Dr. Dre',
 'Amy Winehouse',
 'Corinne Bailey Rae',
 'Shaggy',
 'Beyoncé',
 'Green Day',
 'Foo Fighter

In [6]:
#after we have our list of names
def get_songs_by_artists(playlist_id):
    artist_names = artists_spot(playlist_id)
    all_songs = []

    for artist in artist_names:
        #for every artist from playlist i will search for 40 tracks from artist
        results = sp.search(q='artist:' + artist, type='track', limit=40)
        for track in results['tracks']['items']:
            #appending the artist song and uri
            all_songs.append({
                "artist": artist,
                "song_name": track['name'],
                "uri": track['uri']
            })
            

    df = pd.DataFrame(all_songs)
    return df

In [7]:
df= get_songs_by_artists(playlist_id)

In [8]:
df = df.drop_duplicates(subset='song_name', keep = 'first')
track_uris = df['uri'].str.split(':').str[-1].tolist()

In [9]:
df['artist'].value_counts()

artist
Kanye West             40
Eve                    40
Foster The People      40
Modjo                  39
My Chemical Romance    39
                       ..
Norah Jones            14
Kylie Minogue          14
Coldplay               14
Taio Cruz              12
Nelly Furtado          11
Name: count, Length: 92, dtype: int64

In [10]:
def get_combined_audio_features(track_uris, df):
    
    start = 0
    audio = pd.DataFrame()
    
    for stop in range(0, len(track_uris) + 100, 100):
        if start != stop:
            audio_features = pd.json_normalize(sp.audio_features(track_uris[start:stop]))
            audio = pd.concat([audio_features,audio])
            start = stop
    
    # Merge the audio features DataFrame with the song_info_df on the 'uri' column
    combined_df = pd.merge(df, audio, on='uri', how='right')
    
    return combined_df

In [11]:
combined_df = get_combined_audio_features(track_uris, df)

In [12]:
combined_df['song_name'].value_counts()

song_name
What It's Like                                          1
PRIDE.                                                  1
Sugar (feat. Wynter)                                    1
What A Night (feat. inverness) - Big Game Winner Mix    1
High Heels - Whistle While You Twerk                    1
                                                       ..
Family Affair                                           1
Be Without You - Kendu Mix                              1
Real Love                                               1
Mary Jane (All Night Long)                              1
Papercut                                                1
Name: count, Length: 2773, dtype: int64

### I want to to be able to update the dataframe by adding new playlists and run the same functions and then adding to the existing df

In [43]:
existing_df = pd.read_csv('spotify_files/spotify_df.csv')

In [44]:
existing_df.drop('Unnamed: 0', axis=1, inplace=True)

In [45]:
def add_new_songs(new_playlist_id):
    

    song_df = get_songs_by_artists(new_playlist_id)
    
    song_df = song_df.drop_duplicates(subset='song_name', keep = 'first')
    track_uris = song_df['uri'].str.split(':').str[-1].tolist()
    
    combined_df = get_combined_audio_features(track_uris, song_df)
    
    
    
    return combined_df

In [48]:
new_playlist_id = input('Enter the new playlist ID: ')
combined_df = add_new_songs(new_playlist_id)

Enter the new playlist ID: 3IsxzDS04BvejFJcQ0iVyW


In [49]:
spot_df = pd.concat([existing_df, combined_df], ignore_index = True)
spot_df = spot_df.drop_duplicates(subset= 'song_name', keep = 'first')

In [50]:
spot_df

,artist,song_name,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
0,Eve,What It's Like,spotify:track:6vUPPEN2V2lZ78b6DyEIjZ,0.679,0.557,0,-7.423,1,0.0281,0.090400,0.032400,0.0866,0.405,85.152,audio_features,6vUPPEN2V2lZ78b6DyEIjZ,https://api.spotify.com/v1/tracks/6vUPPEN2V2lZ...,https://api.spotify.com/v1/audio-analysis/6vUP...,303133,4
1,Eve,Kaikai Kitan,spotify:track:6y4GYuZszeXNOXuBFsJlos,0.584,0.911,5,-3.515,0,0.0900,0.011900,0.000000,0.1190,0.606,92.502,audio_features,6y4GYuZszeXNOXuBFsJlos,https://api.spotify.com/v1/tracks/6y4GYuZszeXN...,https://api.spotify.com/v1/audio-analysis/6y4G...,221426,4
2,Eve,Hey Lover,spotify:track:0bhwnn2xqnBUcMZDcXNuII,0.480,0.482,7,-11.450,0,0.0362,0.250000,0.000289,0.3450,0.684,128.160,audio_features,0bhwnn2xqnBUcMZDcXNuII,https://api.spotify.com/v1/tracks/0bhwnn2xqnBU...,https://api.spotify.com/v1/audio-analysis/0bhw...,144000,4
3,Eve,Father Of Mine,spotify:track:2hx4ptqsE8dboLH3NCLmaN,0.568,0.940,1,-5.401,1,0.0411,0.000431,0.000050,0.0779,0.738,97.096,audio_features,2hx4ptqsE8dboLH3NCLmaN,https://api.spotify.com/v1/tracks/2hx4ptqsE8db...,https://api.spotify.com/v1/audio-analysis/2hx4...,231400,4
4,Eve,I See Red,spotify:track:5MAK1nd8R6PWnle1Q1WJvh,0.509,0.448,4,-7.552,0,0.0357,0.007130,0.013700,0.2440,0.221,156.909,audio_features,5MAK1nd8R6PWnle1Q1WJvh,https://api.spotify.com/v1/tracks/5MAK1nd8R6PW...,https://api.spotify.com/v1/audio-analysis/5MAK...,230613,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11862,Gunna,Drip or Drown,spotify:track:6ZthdsKjWtiCxnxbhs74vF,0.952,0.478,5,-8.520,0,0.3610,0.774000,0.000000,0.1120,0.568,118.008,audio_features,6ZthdsKjWtiCxnxbhs74vF,https://api.spotify.com/v1/tracks/6ZthdsKjWtiC...,https://api.spotify.com/v1/audio-analysis/6Zth...,126168,4
11863,Gunna,pushin P (feat. Young Thug),spotify:track:3XOalgusokruzA5ZBA2Qcb,0.773,0.422,1,-4.572,0,0.1870,0.007830,0.006930,0.1290,0.488,77.502,audio_features,3XOalgusokruzA5ZBA2Qcb,https://api.spotify.com/v1/tracks/3XOalgusokru...,https://api.spotify.com/v1/audio-analysis/3XOa...,136267,1
11864,Gunna,Banking On Me,spotify:track:7Fege3yv4dvehB3NdRBGwI,0.793,0.308,0,-10.414,0,0.0466,0.613000,0.000017,0.0953,0.460,135.143,audio_features,7Fege3yv4dvehB3NdRBGwI,https://api.spotify.com/v1/tracks/7Fege3yv4dve...,https://api.spotify.com/v1/audio-analysis/7Feg...,200000,4
11865,Gunna,ca$h $hit,spotify:track:48rsYvIQXUAtxcmIoStOaM,0.795,0.550,7,-5.704,0,0.0882,0.123000,0.000000,0.0873,0.152,119.975,audio_features,48rsYvIQXUAtxcmIoStOaM,https://api.spotify.com/v1/tracks/48rsYvIQXUAt...,https://api.spotify.com/v1/audio-analysis/48rs...,131493,4


In [51]:
spot_df.to_csv('spotify_files/spotify_df.csv')

## While its not perfect i think it does a good job at fetching new songs and adding them to our df.

Steps for use:
start by reading in csv and then run the code, input the playlist uri, run and then save the csv.
you need to read in the csv again to start the loop again.